In [1]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir

    path.append(dir(path[0]))
    __package__ = "dash_app"

In [2]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map
from settings import *

DATA_PATH: /Users/dan/ClimateIreland/CI-Status-Report/CI-Climate-Status-Tool/data/


In [3]:
data_path = DATA_PATH+'Atmospheric_Domain/2.10CarbonDioxide/Figure2.18/'
xls = pd.ExcelFile(
    data_path+'MonthlyMeanConcentrationOfCarbonDioxide_MaunaLoa_MaceHead2018.xlsx')
dataDF = pd.read_excel(xls, 'co2_mm_mlo')
dataDF.rename(columns = {
    "Unnamed: 0":"Date"
    }, inplace = True)
dataDF = dataDF.iloc[:, 0:5]
dataDF[0:20]

,Date,Year,Month,Mauna Loa (Hawaii),Mace Head
0,1958-03-01,1958,3,315.71,NaN
1,1958-04-01,1958,4,317.45,NaN
2,1958-05-01,1958,5,317.50,NaN
3,1958-06-01,1958,6,317.10,NaN
4,1958-07-01,1958,7,315.86,NaN
5,1958-08-01,1958,8,314.93,NaN
6,1958-09-01,1958,9,313.20,NaN
7,1958-10-01,1958,10,312.66,NaN
8,1958-11-01,1958,11,313.33,NaN
9,1958-12-01,1958,12,314.67,NaN


In [4]:
dataDF.dtypes

Date                  datetime64[ns]
Year                           int64
Month                          int64
Mauna Loa (Hawaii)           float64
Mace Head                    float64
dtype: object

In [11]:
MaunaLoa = go.Scatter(x=dataDF["Date"],
                     y=dataDF["Mauna Loa (Hawaii)"],
                     name='Mauna Loa (Hawaii)',
                     line_shape='spline',
                     line=dict(
                            # color="#fc0d1b", color used in report
                            color=TIMESERIES_COLOR_1,
                            width=2),
                      hovertemplate='%{x|%b %Y}<br>' +
                            '<b>Mauna Loa (Hawaii)</b><br>' +
                            'CO₂: %{y:.2f} ppm<extra></extra>' 
                            )

MaceHead = go.Scatter(x=dataDF["Date"],
                     y=dataDF["Mace Head"],
                     name='Mace Head (Ireland)',
                     line_shape='spline',
                     line=dict(
                            # color="#fc0d1b", color used in report
                            color=TIMESERIES_COLOR_2,
                            width=2),
                      hovertemplate='%{x|%b %Y}<br>' +
                            '<b>Mace Head</b><br>' +
                            'CO₂: %{y:.2f} ppm<extra></extra>' 
                            )

In [12]:
figure_2_18 = make_subplots(specs=[[{'secondary_y': False}]])
figure_2_18.add_trace(MaunaLoa,
            secondary_y=False,)
figure_2_18.add_trace(MaceHead,
            secondary_y=False,)

figure_2_18.update_layout(TIMESERIES_LAYOUT)

figure_2_18.update_yaxes(title_text='CO₂ Concentration (ppm)',
                         showgrid=False,
                         fixedrange=True,
                         showspikes=True,
                        )
figure_2_18.update_xaxes(title_text='Year',
                         range=['1955-01-01', '2020-01-01'],
                         tickformat="%Y",
                         showspikes=True,  
                         spikethickness=2
                        )

figure_2_18.show()

In [13]:
# used for static images on main site
figure_2_18.update_layout(
    margin={"t": 40, "b": 20, "r": 40, "l": 80, },
    plot_bgcolor='#f7fbfd',
    paper_bgcolor='white',
    height=500,
    width=900,
    font_size=14,
    title=dict(
        text='<b>Mean Monthly Carbon Dioxide (CO\u2082) Concentration</b>',
        x=0.5,
        y=0.97),)
figure_2_18.update_yaxes(
    secondary_y=False,
    title_standoff=0.2
)
figure_2_18.update_xaxes(
    title_standoff=1
)
figure_2_18.write_image(
    "/Users/dan/ClimateIreland/CI-Status-Report/images/mean__monthly__carbon_dioxide.png", 
    format="png"
    )

In [3]:
data_path = DATA_PATH+'Atmospheric_Domain/2.10CarbonDioxide/Map2.10/'
df = pd.read_csv(data_path+'Map2.10_StationTable.txt', delimiter = ",")
# dataDF["size"]=0
df


,FID,County,Station_Nu,name,Height__m_,Easting,Northing,Latitude,Longitude,Open_Year,Close_Year,Type
0,0,Donegal,1575,Malin_head,20,241939,458562,55.372,-7.339,2009,,EPA
1,1,Galway,275,Mace_Head,21,73400,232276,53.322,-9.901,2003,,NUIG
2,2,Wexford,0,Carnsore Point,9,311656,104080,52.193,-6.376,2005,,EPA
3,3,Kerry,2275,Valentia_Observatory,24,45940,78555,51.936,-10.238,2010,,Synoptic


In [20]:



epaDF = df.loc[(df['Type'] == 'EPA')]
epaDFStr=epaDF.astype(str)
epaTrend = go.Scattermapbox(
    name='EPA',
    lon=epaDF.Longitude,
    lat=epaDF.Latitude,
    marker=dict(color=STATION_COLORS['EPA'],
                size=7),
    hovertemplate='Name: ' + epaDFStr['name'] + '<br>' +
            'County: ' + epaDFStr['name'] + '<br>' +
            'Agency: EPA<br>' +
            'Station No.: ' + epaDFStr['Station_Nu'] + '<br>' +
            'Open Year: ' + epaDFStr['Open_Year'] + '<br>' +
            'Height: ' + epaDFStr['Height__m_'] + ' m<br>' +
            'Lat: %{lat:.2f} \u00b0<br>'+
            'Lon: %{lon:.2f} \u00b0<br><extra></extra>',
)

nuigDF = df.loc[(df['Type'] == 'NUIG')]
nuigDFStr=nuigDF.astype(str)
nuigTrend = go.Scattermapbox(
    name='NUIG',
    lon=nuigDF.Longitude,
    lat=nuigDF.Latitude,
    marker=dict(color=STATION_COLORS['NUIG'],
                size=7),
    hovertemplate='Name: ' + nuigDFStr['name'] + '<br>' +
            'County: ' + nuigDFStr['name'] + '<br>' +
            'Agency: NUIG<br>' +
            'Station No.: ' + nuigDFStr['Station_Nu'] + '<br>' +
            'Open Year: ' + nuigDFStr['Open_Year'] + '<br>' +
            'Height: ' + nuigDFStr['Height__m_'] + ' m<br>' +
            'Lat: %{lat:.2f} \u00b0<br>'+
            'Lon: %{lon:.2f} \u00b0<br><extra></extra>',
)

metDF = df.loc[(df['Type'] == 'Synoptic')]
metDFStr=metDF.astype(str)
metTrend = go.Scattermapbox(
    name='Met Eireann',
    lon=metDF.Longitude,
    lat=metDF.Latitude,
    marker=dict(color=STATION_COLORS['Synoptic'],
                size=7),
    hovertemplate='Name: ' + metDFStr['name'] + '<br>' +
            'County: ' + metDFStr['name'] + '<br>' +
            'Agency: NUIG<br>' +
            'Station No.: ' + metDFStr['Station_Nu'] + '<br>' +
            'Open Year: ' + metDFStr['Open_Year'] + '<br>' +
            'Height: ' + metDFStr['Height__m_'] + ' m<br>' +
            'Lat: %{lat:.2f} \u00b0<br>'+
            'Lon: %{lon:.2f} \u00b0<br><extra></extra>',
)

In [21]:
map_2_10 = go.Figure(data=[epaTrend,nuigTrend,metTrend],
                    layout=MAP_LAYOUT)
map_2_10.update_layout(legend_title="<b>  Agency</b>")
map_2_10